<a href="https://colab.research.google.com/github/JIng-Yu-Huang/generative-AI_hw/blob/main/0415Demo07cAI%E4%BB%A3%E7%90%86%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F_Two_Stage_CoT%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🎯 任務說明：將「員瑛式思考產生器」改寫為 Chain-of-Thought（CoT）推理過程

**原始任務：** 輸入一件小事或倒楣事，生成一段超級正向的社群貼文。

#### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請 LLM 思考五種「為什麼這是一件超級幸運的事」的原因
2. **第二階段（產文階段）**：從這五個原因中挑出最有趣的一個，寫成社群發文（員瑛式風格 + 第一人稱 + emoji + "完全是 Lucky Vicky 呀!" 結尾）

這就是典型的 Planning 模式應用：先拆解、後執行。

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [3]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
#api_key = userdata.get('OpenAI')
#os.environ['OPENAI_API_KEY']=api_key
#provider = "openai"
#model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [5]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: groq
    Found existing installation: groq 0.22.0
    Uninstalling groq-0.22.0:
      Successfully uninstalled groq-0.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

### 2. 使用 AISuite 的準備

In [6]:
import aisuite as ai

In [7]:
provider_planner = "groq"
model_planner="llama3-70b-8192"

provider_writer = "groq"
model_writer = "llama3-70b-8192"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [8]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [13]:
system_planner = "請翻譯成台灣習慣的中文回應。你是一位正向思考導師, 擅長將倒楣的小事轉化為幸運的觀點。請針對使用者的事件，思考出五種『為什麼這是超幸運的事』的原因, 列出條列式清單。用第一人稱思考也可以"
system_writer = "請翻譯成台灣習慣的中文回應。你是一位超級樂觀的社群貼文小天使，風格活潑好笑，善於用正向觀點撰寫貼文。請根據一個特別有趣的理由，寫出一段 Instagram 發文口吻的第一人稱貼文。結尾一定要說：『完全是 Lucky Vicky 呀！』而且中間要加入 emoji。"

In [14]:
def lucky_post(prompt):
    # Step 1: CoT - 思考五種幸運理由
    planning_prompt = f"使用者說：{prompt}。請幫我想五種這是超級幸運的理由。"
    lucky_reasons = reply(system_planner, planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    # Step 2: 選出最有趣一項，寫成貼文
    generation_prompt = f"這是我想到的五個理由：\n{lucky_reasons}\n\n請從中選出一個最有趣的理由，然後根據它寫一段員瑛式發文。"
    lucky_post = reply(system_writer, generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return lucky_reasons, lucky_post

In [15]:
lucky_post("今天教授叫我找好多論文")

('I totally get it! 😊 As a positive thinking coach, I\'d be delighted to help you shift your perspective on this situation. Here are five reasons why getting asked to find many research papers is actually a super lucky thing:\n\n**1. 這是一個學習的機會！**\nI get to learn so much from reading all these research papers! I\'ll gain valuable knowledge and insights in my field, which will benefit me in the long run. This is a chance to upgrade my skills and become an expert in my area of interest.\n\n**2. 我的教授信任我！**\nMy professor must trust me to give me this task! It means they believe in my abilities and think I can handle the responsibility. This is a great confidence booster and a reminder that my hard work is recognized.\n\n**3. 這是一個鍛鍊時間管理的機會！**\nI\'ll get to practice my time management skills, prioritize tasks, and become more efficient in completing assignments. These skills will benefit me not only in academics but also in my future career.\n\n**4. 我會發現新的研究方向！**\nWhile reading through these 

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [16]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00


In [17]:
import gradio as gr

In [18]:
with gr.Blocks() as demo:
    gr.Markdown("### 🍀 員瑛式思考產生器 Lucky Vicky 🌈")
    gr.Markdown("請輸入一件你覺得超小事，甚至有點倒楣的事，讓 AI 幫你用超正向的方式重新詮釋！")
    user_input = gr.Textbox(label="今天發生的事情是…")
    btn = gr.Button("生成 Lucky Vicky 的神級貼文 ✨")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 五個超幸運的理由（Planning CoT）")
        out2 = gr.Textbox(label="📣 最終員瑛式貼文（Po 文）")

    btn.click(lucky_post, inputs=[user_input], outputs=[out1, out2])

In [19]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6282318cb2f6a6b972.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6282318cb2f6a6b972.gradio.live
